In [22]:
pip install scipy


Note: you may need to restart the kernel to use updated packages.Defaulting to user installation because normal site-packages is not writeable




[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
گل‌های زنبق
</font>
</h1>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
در این تمرین، می‌خواهیم به پیاده‌سازی الگوریتمی به نام «k نزدیک ترین همسایه» یا <a href="https://fa.wikipedia.org/wiki/%D8%A7%D9%84%DA%AF%D9%88%D8%B1%DB%8C%D8%AA%D9%85_%DA%A9%DB%8C-%D9%86%D8%B2%D8%AF%DB%8C%DA%A9%E2%80%8C%D8%AA%D8%B1%DB%8C%D9%86_%D9%87%D9%85%D8%B3%D8%A7%DB%8C%D9%87" target="_blank" style="color:#0099cc">k-NN</a> بپردازیم که از الگوریتم‌های رایج یادگیری ماشین به حساب می‌آید. با انجام این تمرین، موضوعاتی چون محاسبات برداری به کمک پخش آرایه‌ها، استفاده از توابع تجمیعی و عمومی نامپای و نمایه‌سازی به کمک آرایه‌ها را مرور خواهیم کرد.
</font>
</p>

In [23]:
import numpy as np

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
مجموعه‌داده
</font>
</h2>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
در این تمرین یک مجموعه‌داده‌ی معروف به نام <a href="https://en.wikipedia.org/wiki/Iris_flower_data_set" target="_blank" style="color:#0099cc">Iris Flower</a> (گل‌های زنبق) در اختیارتان قرار گرفته که شامل مشخصات ۱۵۰ گل زنبق است. هرکدام از این گل‌ها به یکی از سه نوع زنبق نوک‌زبر (Iris setosa)، زنبق رنگارنگ (Iris versicolor) و زنبق ویرجینیا (Iris virginica) تعلق دارد. همچنین مشخصات هر گل شامل طول و عرض کاسبرگ (sepal) و گلبرگ (petal) است. با استفاده از این مشخصات، می‌توانیم هر گل را به صورت یک نقطه در فضایی چهاربعدی تصور کنیم که این چهار کمیت، مختصات آن نقطه را تعیین می‌کنند. 
</font>
</p>


<center>
<div style="max-width: 900px">
<img src="img/iris_types.jpg">
</div>
</center>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
مشخصات ۱۲۰ گل زنبق در فایلی به‌نام <code>irises.npy</code> ذخیره شده است. هنگامی‌که این فایل خوانده شود، یک آرایه‌ی ۱۲۰ در ۴ همانند شکل زیر ساخته می‌شود که هر سطر آن، مشخصات یک گل (طول کاسبرگ، عرض کاسبرگ، طول گلبرگ و عرض گلبرگ به سانتی‌متر) را نشان می‌دهد.
</font>
</p>

<center>
<div style="max-width: 700px">
<img src="img/fig_irises.png">
</div>
</center>

In [24]:
irises = np.load('irises.npy')
print(irises.shape)

(120, 4)



<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
از طرفی نوع این گل‌ها (نوک‌زبر، رنگارنگ یا ویرجینیا) به‌صورت عددی در فایل دیگری به نام <code>types.npy</code> ذخیره شده است. مقادیر این آرایه به‌صورت <code>0</code>، <code>1</code> یا <code>2</code> است که به‌ترتیب بیان‌گر نوع زنبق نوک‌زبر، زنبق رنگارنگ و زنبق ویرجینیا هستند.
</font>
</p>

In [25]:
types = np.load('types.npy')
print(types.shape)

(120,)


<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
در این تمرین قرار است الگوریتمی را پیاده‌سازی کنیم که با دریافت مشخصات یک گل زنبق،‌ نوع آن را پیش‌بینی کند. این پیش‌بینی، به کمک نمونه‌هایی انجام خواهد گرفت که نوع‌شان را از قبل می‌دانیم، یعنی همان داده‌هایی که تا قبل از این خواندید. از این پس چنین نمونه‌هایی را <b style="color:#0099cc">نمونه‌های آموزشی (train)</b> خواهیم خواند.
<br>
مشخصات مربوط به ۳۰ گل زنبق دیگر که نوع‌شان را <b>نمی‌دانیم</b> و قرار است به کمک الگوریتم k-NN، نوع آن‌ها را پیش‌بینی کنیم در فایل دیگری به نام <code>new_irises.npz</code> در اختیارتان قرار گرفته است. هنگامی‌که این فایل خوانده شود، یک آرایه‌ی ۳۰ در ۴ همانند شکل زیر ساخته می‌شود که هر سطر آن، مشخصات یک گل را نشان می‌دهد. چنین نمونه‌هایی را <b style="color:#0099cc">نمونه‌های آزمون (test)</b> خواهیم خواند.
</font>
</p>

<center>
<div style="max-width: 700px">
<img src="img/fig_new_irises.png">
</div>
</center>

In [26]:
new_irises = np.load('new_irises.npy')
print(new_irises.shape)

(30, 4)


<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
تاکنون با موفقیت تمام فایل‌های مجموعه‌داده را خواندیم. اکنون تعداد نمونه‌های آموزشی را <code>n</code> و تعداد نمونه‌های آزمون را <code>m</code> بنامیم و در متغیرهایی به همین نام ذخیره کنیم.
</font>
</p>

In [27]:
n, m = len(irises), len(new_irises)
print("Number of training samples (n):", n)
print("Number of test samples (m):", m)

Number of training samples (n): 120
Number of test samples (m): 30


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
الگوریتم k نزدیک‌ترین همسایه
</font>
</h2>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
فرض کنید مشخصات یک گل زنبق به شما داده شده و قرار است نوع آن را تشخیص دهید. احتمالاً اولین ایده‌ای که به ذهن‌تان می‌رسد این است که شباهت آن را با گل‌هایی که نوع‌شان را می‌دانید بسنجید و براساس این شباهت، نوع آن را پیش‌بینی کنید. فرض کنید ۳ گل که بیشترین شباهت به این گل داشته‌اند را پیدا کرده‌اید. حال می‌دانید که از این ۳ گل، مثلاً ۲ گل از نوع ویرجینیا است و ۱ گل از نوع نوک‌زبر. حتماً شما هم موافقید که طبق رای اکثریت، احتمال این‌که گل ما هم از نوع ویرجینیا باشد بیشتر است. پس همین نوع را به‌عنوان پیش‌بینی خود اعلام می‌کنیم.
<br>
این دقیقاً همان کاری‌ست که الگوریتم k-NN انجام می‌دهد. البته در این الگوریتم معمولاً به جای پیدا کردن بیشترین شباهت، کمترین فاصله (تفاوت) محاسبه می‌شود. مراحل این الگوریتم به شرح زیر است: 

<ol dir=rtl>
<li>ابتدا فاصله‌ی نمونه‌ی آزمون (نمونه‌ای که قصد پیش‌بینی نوع آن را داریم) را با تمام نمونه‌های آموزشی (نمونه‌هایی که نوع آن‌ها را می‌دانیم) محاسبه می‌کنیم.</li>
<li>سپس <code>k</code> نمونه‌ی آموزشی که کمترین فاصله را با نمونه‌ی آزمون ما داشته‌اند پیدا می‌کنیم.</li>
<li>اکنون بررسی می‌کنیم که این <code>k</code> نمونه، بیشتر متعلق به کدام نوع بوده‌اند؟ همان نوع را به‌عنوان پیش‌بینی خود برای نمونه‌ی آزمون اعلام می‌کنیم.</li>
</ol>
</font>
</p>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
اکنون شروع کنیم و هرکدام از گام‌های این الگوریتم را به کمک نامپای پیاده‌سازی کنیم.

</font>
</p>

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
قسمت ۱: محاسبه‌ی فاصله‌ها
</font>
</h2>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
در ابتدا قصد داریم تابعی بنویسیم که فاصله‌ی هرکدام از نمونه‌ها/ردیف‌های یک آرایه همچون <code>new_points</code> را با هرکدام از نمونه‌ها/ردیف‌های آرایه‌ی دیگر همچون <code>points</code> محاسبه کند. به‌عنوان مثال در این تمرین اگر ما ۱۲۰ نمونه‌ی آموزشی و ۳۰ نمونه‌ی آزمون داریم، باید فاصله‌ی هرکدام از نمونه‌های آزمون با هرکدام از نمونه‌های آموزشی را محاسبه کنیم، بنابراین خروجی ما دارای ابعاد ۳۰ در ۱۲۰ خواهد بود. به‌عنوان مثال ردیف اول این ماتریس که در شکل سمت راست زیر نشان داده شده،‌ فاصله‌ی نمونه‌ی آزمون اول را با هرکدام از نمونه‌های آموزشی شامل می‌شود.
</font>
</p>

<center>
<div style="max-width: 1000px">
<img src="img/fig.png">
</div>
</center>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
برای محاسبه‌ی فاصله‌ی بین دو نقطه‌ی <code>p</code> و <code>q</code> با <code>f</code> ویژگی، از رابطه‌ی زیر استفاده خواهیم کرد:
</font>
</p>

$$ d(p,q) = {\sum_{i=0}^{f-1} (p_i - q_i)^2} $$

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
در تمرین ما،‌ <code>f</code> برابر با <code>4</code> است. یعنی برای محاسبه‌ی فاصله‌ی بین دو نمونه کافیست عرض کاسبرگ آن‌ها را از هم کم کرده و به توان ۲ برسانیم، سپس  برای طول کاسبرگ،‌ عرض گلبرگ و طول گلبرگ نیز همین کار را انجام داده و حاصل به‌دست‌آمده برای هر چهار ویژگی را با هم جمع کنیم. 
برای محاسبه‌ی فاصله‌ی بین اجزای دو آرایه روش‌های مختلفی وجود دارد که ۳ رویکرد مختلف را آزمایش خواهیم کرد.
</font>
</p>

<h3 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
روش دو حلقه‌ای
</font>
</h3>


<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
ابتدا می‌خواهیم این کار را به‌سادگی و به کمک دو حلقه (<code>for</code>) انجام دهیم. یعنی در این روش یک حلقه روی نمونه‌های آزمون می‌نویسیم و داخل آن یک حلقه روی نمونه‌های آموزشی می‌نویسیم تا به ازای هر نمونه‌ی آزمون،‌ مقدار فاصله با هرکدام از نمونه‌های آموزشی محاسبه شود.
<br>
در تابع زیر کافیست عبارتی را جلوی علامت مساوی (به‌جای <code>None</code>) قرار دهید که فاصله‌ی بین دو نقطه‌ی <code>new_points[i]</code> و <code>points[j]</code> را به کمک فرمول بالا محاسبه کند.
<br>
<b style="color:green">راهنمایی:</b>
پیشنهاد می‌کنیم در این قسمت تنها از توابع مجذور (<code>np.square</code>) و مجموع (<code>np.sum</code>) خود نامپای استفاده کنید.
</font>
</p>

In [28]:
def calc_two_loops(new_points, points):
    
    #‌ m is the number of new points (test samples)
    m = len(new_points)
    # n is the number of points (training samples)
    n = len(points)
    # Distance matrix
    d = np.zeros((m, n))
    
    # For each new point
    for i in range(m):
        # For each point
        for j in range(n):
            # Calculate the distance between the two points
            d[i, j] = np.sqrt(np.sum(np.square(points[j]-new_points[i]))) # TO-DO
            
    return d

In [29]:
d2 = calc_two_loops(new_irises, irises)
print(d2.shape)

(30, 120)


<h3 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
روش تک حلقه‌ای
</font>
</h3>


<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
اکنون می‌خواهیم از خاصیت پخش یا بازآرایی آرایه‌ها استفاده کرده و به‌ترتیب هرکدام از حلقه‌ها را حذف کنیم. در این قسمت می‌خواهیم به کمک خاصیت پخش آرایه، حلقه‌ی داخلی را حذف کرده و فاصله‌ی هر نمونه‌ی آزمون را به‌صورت همزمان با همه‌ی نمونه‌های آموزشی محاسبه کنیم. پس در تابع زیر، این‌بار جلوی علامت مساوی (به‌جای <code>None</code>) عبارتی را قرار دهید که فاصله‌ی همه‌ی نمونه‌های آموزشی با نمونه‌ی آزمون <code>new_points[i]</code> را محاسبه کند.
<br>
<b style="color:purple">نکته:</b>
خروجی این قسمت دقیقاً همانند خروجی تابع دو حلقه‌ای است، با این تفاوت که در تابع قبلی در هر مرحله، مقدار یک عنصر از ماتریس محاسبه می‌شد اما در این قسمت در هر مرحله، یک ردیف کامل از ماتریس محاسبه می‌شود.
</font>
</p>

In [30]:
def calc_one_loop(new_points, points):
    
    #‌ m is the number of new points (test samples)
    m = len(new_points)
    # n is the number of points (training samples)
    n = len(points)
    # Distance matrix
    d = np.zeros((m, n))
    
    # For each new point
    for i in range(m):
        # Calculate the distance between the new point and all the points
        d[i] = np.sqrt(np.sum(np.square(new_points[i] - points), axis=1)) # TO-DO
        
    return d

In [31]:
d1 = calc_one_loop(new_irises, irises)
print(d1.shape)

(30, 120)


<h3 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
روش بدون حلقه
</font>
</h3>


<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
این‌بار باز هم از خاصیت‌های آرایه‌ها استفاده کرده و در تابع زیر جلوی <code>return</code> تنها یک خط کد بنویسید که کل فاصله‌ها را خروجی دهد.
<br>
ممکن است ساختن آرایه‌ی فاصله‌ها بدون حلقه، پیچیده به نظر برسد. اما استفاده از توابع نامپای به جای استفاده از حلقه، هم باعث سریع‌تر شدن کد و هم کوتاه‌تر شدن آن می‌شود و خوب است به آن عادت کنید. سریع‌تر شدن کد را می‌توانید با ورودی دادن آرایه‌های بزرگ آزمایش کنید.
</font>
</p>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
<b style="color:green">راهنمایی ۱:</b> یک راه‌حل ممکن این است که ماتریس <code>new_points</code> را به گونه‌ای بازآرایی کنیم که به شکل <code>m</code> آرایه‌ با ابعاد <code dir=ltr>(1, f)</code> در بیاید (<code>f</code> تعداد ویژگی‌هاست که در این تمرین برابر <code>4</code> است) و <code>points</code> هم به <code>1</code> آرایه با ابعاد <code>(n, f)</code> تبدیل شود. پس از آن کافیست به ترتیب به محاسبه‌ی فاصله‌ها، مجذور (توان ۲) فاصله‌ها و در نهایت مجموع بپردازیم. توجه داشته باشید که تمام این موارد باید در یک خط کد انجام شوند.
</font>
</p>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
<b style="color:green">راهنمایی ۲:</b> برای بازآرایی می‌توانید از <code>reshape</code> استفاده کنید،‌ یا این‌که از <code>np.newaxis</code> که در درسنامه‌ی نمایه‌سازی با آرایه‌ها معرفی شد کمک بگیرید.
</font>
</p>

In [32]:
def calc_no_loop(new_points, points):
    return np.sqrt(np.sum((np.reshape(points, (1,120, 4))-np.reshape(new_points, (30, 1, 4)))**2, axis=2))

In [33]:
d = calc_no_loop(new_irises, irises)
print(d.shape)

(30, 120)


<h3 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
بررسی
🧐
</font>
</h3>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
محاسبه‌ی آرایه فاصله‌ها به سه روش فوق، نباید تفاوتی در نتیجه حاصل کند. از این رو کد زیر را اجرا کنید تا از برابر بودن خروجی سه تابع فوق اطمینان حاصل کنید. از این به بعد از آرایه‌ی ‍<code>d</code> به عنوان آرایه‌ی فاصله‌ها استفاده می‌کنیم.
</font>
</p>

In [34]:
if np.allclose(d, d1, 1e-5) and np.allclose(d, d2, 1e-5) and np.allclose(d1, d2, 1e-5):
    print('Fine!')
else:
    print('There is something wrong!')

Fine!


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
قسمت ۲: پیدا کردن <code>k</code> نزدیک‌ترین همسایه
</font>
</h2>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
    به کمک توابع پیاده‌سازی شده توانستیم فاصله‌ی هر نمونه‌ی آزمون با هرکدام از نمونه‌های آموزشی را محاسبه کنیم و حاصل را در <code>d</code> ذخیره کردیم.
    یعنی فاصله‌ی نمونه‌ی آزمون <code>new_irises[i]</code> با نمونه‌ی آموزشی <code>irises[j]</code> را در درایه‌ی <code>d[i][j]</code> ذخیره کرده‌ایم. اکنون نیاز است به‌ازای هر نمونه‌ی آزمون، <code>k</code> نمونه‌ی آموزشی با کمترین فاصله به آن را پیدا کنیم. توجه داشته باشید که باید شماره‌ی نزدیک‌ترین نمونه‌ها را ذخیره کنیم، نه خود مقدار فاصله را. در اینجا <code>k</code> را برابر <code>10</code> در نظر بگیرید، بنابراین آرایه‌ی خروجی <code>k_nearest</code> یک ماتریس با ابعاد <code dir=ltr>(30, 10)</code> خواهد بود.
</font>
</p>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
    <b style="color:green">راهنمایی:</b>
    به منظور پیدا کردن نزدیک‌ترین همسایگان (کم‌ترین فاصله‌ها) از <code>np.argpartition</code> استفاده کنید.
    تفاوت <code>argpartition</code> با <code>partition</code> همچون تفاوت <code>argsort</code> با <code>sort</code> است. یعنی این تابع، شماره (اندیس) عناصر مرتب‌شده را خروجی می‌دهد و نه مقدار آن‌ها را.
</font>
</p>

In [35]:
k = 10
k_nearest = np.argpartition(d, k)[:,:k]
print(k_nearest.shape)

(30, 10)


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
قسمت ۳: پیدا کردن نوع <code>k</code> نزدیک‌ترین همسایه
</font>
</h2>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
    اکنون اندیس نزدیک‌ترین همسایه‌ها (نمونه‌های آموزشی) برای هر نمونه‌ی آزمون را در آرایه‌ی <code>k_nearest</code> داریم. اما ما به دنبال نوع هرکدام از آن‌ها هستیم. برای آن‌که به‌جای هرکدام از اندیس‌ها،‌ نوع آن گل را جایگزین کنید می‌توانید از آرایه‌ی <code>types</code> استفاده کنید. این کار را در یک خط انجام داده و متغیر ‌<code>k_nearest_types</code> را مقداردهی کنید.
</font>
</p>

In [36]:
k_nearest_types = types[k_nearest]
k_nearest_types

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 2, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 2],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
       [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
       [2, 2, 2, 2, 2, 2, 2, 2, 2, 1],
       [1, 1, 1, 2, 2, 2, 2, 2, 2, 1],
       [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
       [2, 2, 2, 2, 2, 2,

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
قسمت ۴: تعیین نوع گل‌های جدید
</font>
</h2>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
    در نهایت، باید یک آرایه به نام <code>predicted_types</code> و به طول <code>m</code> بسازیم که در خانه‌ی <code>predicted_types[i]</code>، پرتکرارترین شناسه در <code>k_nearest_types[i]</code> به عنوان شناسه نوع پیش‌بینی شده برای <code>new_irises[i]</code> قرار می‌گیرد.
    <br>
    از آنجا که نامپای مستقیماً‌ تابع مُد (<code>mode</code>) ندارد، این کار را به کمک تابع <code>mode</code> که داخل کتابخانه‌ی <code>scipy</code> قرار دارد، انجام داده‌ایم.
</font>
</p>

In [37]:
from scipy import stats
predicted_types = stats.mode(k_nearest_types, axis=1).mode.reshape(m)
predicted_types

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2])

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
در این قسمت، آرایه‌ی <code>new_types</code> بارگیری شده است که حاوی جواب‌های درست است. از روی آن، دقت دسته‌بندی‌ را محاسبه و در متغیر <code>accuracy</code> ذخیره کنید. دقت دسته‌بندی برابر با تعداد پیش‌بینی‌های درست تقسیم بر تعداد نمونه‌های آزمون است. اگر مراحل کار را درست انجام داده باشید، دقت باید برابر ۱ باشد (یعنی دسته‌بند نوع تمام گل‌های آزمون را درست پیش‌بینی کرده است).
</font>
</p>

In [42]:
new_types = np.load('new_types.npy')
accuracy = 1 - (np.count_nonzero(new_types-predicted_types) / 30)
print('Accuracy:', accuracy)

Accuracy: 1.0


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
<b>سلول جواب‌ساز</b>
</font>
</h2>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
    برای ساخته‌شدن فایل <code>result.zip</code> سلول زیر را اجرا کنید. توجه داشته باشید که پیش از اجرای سلول زیر تغییرات اعمال شده در نت‌بوک را ذخیره کرده باشید (<code>ctrl+s</code>) تا در صورت نیاز به پشتیبانی امکان بررسی کد شما وجود داشته باشد.
</font>
</p>

In [43]:
import zipfile
from inspect import getsource

np.savez("answers.npz", k_nearest = k_nearest, k_nearest_types = k_nearest_types, predicted_types = predicted_types)

def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)

imp = 'import numpy as np\n'
%save calc_two_loops imp + getsource(calc_two_loops)
%save calc_one_loop imp + getsource(calc_one_loop)
%save calc_no_loop imp + getsource(calc_no_loop)

file_names = ["calc_two_loops.py", "calc_one_loop.py", "calc_no_loop.py",
              "answers.npz", "iris_flowers.ipynb"]
compress(file_names)

The following commands were written to file `calc_two_loops.py`:
import numpy as np
def calc_two_loops(new_points, points):
    
    #‌ m is the number of new points (test samples)
    m = len(new_points)
    # n is the number of points (training samples)
    n = len(points)
    # Distance matrix
    d = np.zeros((m, n))
    
    # For each new point
    for i in range(m):
        # For each point
        for j in range(n):
            # Calculate the distance between the two points
            d[i, j] = np.sqrt(np.sum(np.square(points[j]-new_points[i]))) # TO-DO
            
    return d

The following commands were written to file `calc_one_loop.py`:
import numpy as np
def calc_one_loop(new_points, points):
    
    #‌ m is the number of new points (test samples)
    m = len(new_points)
    # n is the number of points (training samples)
    n = len(points)
    # Distance matrix
    d = np.zeros((m, n))
    
    # For each new point
    for i in range(m):
        # Calculate the dista